We are uploading all csv files of the Scarlet & Violet main and special sets, then merging all 13 sets into a master set.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [1]:
import pandas
